Create an application that should be used by the HR Team to filter the resume based on the Skills.
 We are going to use a dataset from kaggle and then to clean and pre-process it to fit a suitable classification model in order to assist the HR team
 to filter resume based on the job-role. 

In [3]:
import spacy #using spacy module for vectorization
nlp= spacy.load('en_core_web_lg') #en_core_web_lg is the pre-trained model by spacy

In [31]:
import pandas as pd
import numpy as np
import re
import string
import unidecode

In [49]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mogit\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
df=pd.read_csv("UpdatedResumeDataSet.csv") #dataset downloaded from the kaggle "https://www.kaggle.com/datasets/jillanisofttech/updated-resume-dataset"

In [11]:
df.shape

(962, 2)

In [12]:
df.head()


,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [54]:
df.Category.value_counts()

Java Developer               84
Testing                      70
DevOps Engineer              55
Python Developer             48
Web Designing                45
HR                           44
Hadoop                       42
Blockchain                   40
ETL Developer                40
Operations Manager           40
Data Science                 40
Sales                        40
Mechanical Engineer          40
Arts                         36
Database                     33
Electrical Engineering       30
Health and fitness           30
PMO                          30
Business Analyst             28
DotNet Developer             28
Automation Testing           26
Network Security Engineer    25
SAP Developer                24
Civil Engineer               24
Advocate                     20
Name: Category, dtype: int64

In [50]:
#we need to pre-process and clean the data
def clean_words(text):
    """Basic cleaning of texts"""
    
    # remove html
    text=re.sub("(<.*?>)","",text)
    
    #remove non-ascii and digits
    text=re.sub("(\\W|\\d)"," ",text)
    
    #remove whitespace
    text=text.strip()
    
    #removing single charcters pattern
    text=re.sub(pattern='\s+[a-zA-Z]\s+' ,repl=" ", string=text)
    
    #remove accented characters
    text=unidecode.unidecode(text) #we have accented characters like a^ etc, so to remove that we are performing 
    
    #to make words into lowercase
    text=text.lower()
    
    #removing stop words from the paragraph
    words = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    text = " ".join(words)
    
    #here we are avoiding tokenization since the spacy's model takes string or doc as input not list of words
    # also we are not performing stemming and lemmatization since it will change the context of skills and other words in resume text
    
    
    return text

In [56]:
df['cleaned_text']=df.Resume.map(lambda x: clean_words(x))

In [57]:
df.head()

,Category,Resume,cleaned_text
0,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas num...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education details may may e uit rgpv data scie...
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",areas interest deep learning control system de...
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skills python sap hana tableau sap hana sql sa...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education details mca ymcaust faridabad haryan...


Now that we have cleaned the text it is ready to vectorize

In [58]:
df['vectorized_data']=df.cleaned_text.apply(lambda text: nlp(text).vector)

In [59]:
df.head()

,Category,Resume,cleaned_text,vectorized_data
0,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas num...,"[0.00019259728, -0.063411176, 0.022081006, 0.5..."
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education details may may e uit rgpv data scie...,"[-0.38565177, 0.41584927, -0.22689901, 0.05342..."
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",areas interest deep learning control system de...,"[-0.34571993, 0.064293616, -0.47398934, 0.1207..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skills python sap hana tableau sap hana sql sa...,"[-0.06141665, -0.009636918, -0.60103905, 0.345..."
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education details mca ymcaust faridabad haryan...,"[0.102590054, -0.23031569, -0.4390396, -0.2980..."


The pre-processing and vectoriztion part is done, Now we will proceed with fitting a best model for the dataset and check if it accurately predicts the results

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [61]:
x=df['vectorized_data']
y=df['Category']

In [65]:
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.3,
    random_state=1049
)

In [72]:
knn=KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean') #using 5 as K 

In [73]:
#note that the vectorized array using spacy model will be an numpy array within numpy array (3D), so inorder to make it 2D numpy array which suits for analysis we need to flatten it out
x_train=np.stack(x_train)
x_test=np.stack(x_test)

In [74]:
knn.fit(x_train,y_train)
y_predtest=knn.predict(x_test)

In [79]:
print(classification_report(y_test, y_predtest))

                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         6
                     Arts       0.80      1.00      0.89         8
       Automation Testing       1.00      0.50      0.67         4
               Blockchain       1.00      1.00      1.00        10
         Business Analyst       0.57      0.80      0.67         5
           Civil Engineer       1.00      0.50      0.67        10
             Data Science       1.00      1.00      1.00        15
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       0.77      0.91      0.83        11
         DotNet Developer       1.00      0.82      0.90        11
            ETL Developer       0.79      1.00      0.88        11
   Electrical Engineering       0.75      1.00      0.86        12
                       HR       1.00      0.47      0.64        15
                   Hadoop       1.00      1.00      1.00     

#Now we will check fitting the data with an ensemble classification model Gradient boosting classifier and check the accuracy and F1 score

In [85]:
from sklearn.ensemble import GradientBoostingClassifier

In [86]:
gbc=GradientBoostingClassifier(learning_rate=0.1,n_estimators=100)

In [87]:
gbc.fit(x_train,y_train)
y_predtestgbc=gbc.predict(x_test)

In [88]:
print(classification_report(y_test, y_predtestgbc))

                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         6
                     Arts       1.00      1.00      1.00         8
       Automation Testing       1.00      1.00      1.00         4
               Blockchain       1.00      1.00      1.00        10
         Business Analyst       1.00      1.00      1.00         5
           Civil Engineer       1.00      1.00      1.00        10
             Data Science       1.00      0.73      0.85        15
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.91      0.95        11
         DotNet Developer       1.00      1.00      1.00        11
            ETL Developer       1.00      1.00      1.00        11
   Electrical Engineering       1.00      1.00      1.00        12
                       HR       1.00      1.00      1.00        15
                   Hadoop       1.00      1.00      1.00     

#wow ! the results were great when compared to KNN